In [1]:
import numpy as np 
import pandas as pd 
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/kaggle/input/santandar-processed-data/fdata.csv


In [2]:
train = pd.read_csv("/kaggle/input/santandar-processed-data/fdata.csv")

In [3]:
cols_to_remove = ['employee_index', 'country_residence', 'new_customer_index', 'primary_customer', 
                  'customer_type_beginning_month', 'customer_relation_type_beginning_month', 
                  'residence_index', 'foreigner_index','registration_date']

train = train.drop(cols_to_remove, axis=1)


In [4]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder

# 1. Convert date columns to seconds
train['fecha_alta']=train['data_date']
train['data_date'] = pd.to_datetime(train['data_date']).astype(int) / 10**9


# 2. Encode object columns to integer values using label encoder
encoder = LabelEncoder()
train['channel_used'] = encoder.fit_transform(train['channel_used'])
train['province_name'] = encoder.fit_transform(train['province_name'])
train['segmentation'] = encoder.fit_transform(train['segmentation'])


In [5]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13618663 entries, 0 to 13618662
Data columns (total 35 columns):
 #   Column                        Dtype  
---  ------                        -----  
 0   data_date                     float64
 1   gender                        int64  
 2   seniority                     float64
 3   channel_used                  int64  
 4   province_code                 float64
 5   province_name                 int64  
 6   activity_index                float64
 7   gross_income_household        float64
 8   segmentation                  int64  
 9   prod_savings_account          int64  
 10  prod_guarantees               int64  
 11  prod_current_accounts         int64  
 12  prod_derivative_account       int64  
 13  prod_payroll_account          int64  
 14  prod_junior_account           int64  
 15  prod_mas_particular_account   int64  
 16  prod_particular_account       int64  
 17  prod_particular_plus_account  int64  
 18  prod_short_term_depo

In [6]:
train.head()

,data_date,gender,seniority,channel_used,province_code,province_name,activity_index,gross_income_household,segmentation,prod_savings_account,...,prod_loans,prod_taxes,prod_credit_card,prod_securities,prod_home_account,prod_payroll,prod_pensions2,prod_direct_debit,age_group,fecha_alta
0,1.422403e+09,0,6.0,153,29.0,31,1.0,87218.10,1,0,...,0,0,0,0,0,0.0,0.0,0,1,2015-01-28
1,1.422403e+09,1,35.0,150,13.0,16,0.0,35548.74,0,0,...,0,0,0,0,0,0.0,0.0,0,1,2015-01-28
2,1.422403e+09,1,35.0,150,13.0,16,0.0,122179.11,0,0,...,0,0,0,0,0,0.0,0.0,0,1,2015-01-28
3,1.422403e+09,0,35.0,149,50.0,51,0.0,119775.54,0,0,...,0,0,0,0,0,0.0,0.0,0,1,2015-01-28
4,1.422403e+09,1,35.0,150,50.0,51,1.0,101850.00,0,0,...,0,0,0,0,0,0.0,0.0,0,1,2015-01-28


In [7]:
train.iloc[0]

data_date                       1422403200.0
gender                                     0
seniority                                6.0
channel_used                             153
province_code                           29.0
province_name                             31
activity_index                           1.0
gross_income_household               87218.1
segmentation                               1
prod_savings_account                       0
prod_guarantees                            0
prod_current_accounts                      1
prod_derivative_account                    0
prod_payroll_account                       0
prod_junior_account                        0
prod_mas_particular_account                0
prod_particular_account                    0
prod_particular_plus_account               0
prod_short_term_deposits                   0
prod_medium_term_deposits                  0
prod_long_term_deposits                    0
prod_e_account                             0
prod_funds

# Popular Recommendation System

In [8]:
import pandas as pd

# Define the popularity metric: Purchasing Amount as the single metric
popular_df = pd.DataFrame(columns=['product', 'sales_volume', 'sales_frequency'])

data_to_concat = []

for i in train.columns[18:-2]:
    sales_volume = train[i].value_counts().get(1, 0)  # Get sales volume, handle case where value 1 is not present
    sales_frequency = round(sales_volume / train.shape[0], 2)  # Calculate sales frequency
    data_to_concat.append({'product': i, 'sales_volume': sales_volume, 'sales_frequency': sales_frequency})

popular_df = pd.concat([popular_df, pd.DataFrame(data_to_concat)], ignore_index=True)

popular_df.sort_values('sales_frequency', inplace=True, ascending=False)
popular_df

/tmp/ipykernel_24/474524568.py:13: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  popular_df = pd.concat([popular_df, pd.DataFrame(data_to_concat)], ignore_index=True)


,product,sales_volume,sales_frequency
14,prod_direct_debit,1744771,0.13
3,prod_e_account,1128003,0.08
13,prod_pensions2,809832,0.06
12,prod_payroll,745728,0.05
8,prod_taxes,706788,0.05
2,prod_long_term_deposits,586204,0.04
9,prod_credit_card,605550,0.04
10,prod_securities,349306,0.03
4,prod_funds,252155,0.02
6,prod_pensions1,125077,0.01


# Enhanced Popularity-based Product Recommender with Time Decay Adjustment

In [9]:
# Try adding Time Decay impact on the Popularity-based Recommendation:
from math import exp

popular_df_time = train.iloc[:,[0,1,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45]]
popular_df_time['year'] = popular_df_time['fecha_dato'].dt.year
popular_df_time['month'] = popular_df_time['fecha_dato'].dt.month
popular_df_time['day'] = '01'

# For computational ease, using the 2016 data ONLY
#popular_df_time = popular_df_time[popular_df_time['year'] == 2016]

# Group by customer id, year and month 
popular_df_time['year_month'] = pd.to_datetime(popular_df_time[['year','month','day']])
popular_df_time.drop(['cust_id','year','month','day'], axis = 1, inplace = True)
popular_df_time_g = popular_df_time.groupby(['year_month'], as_index = False).sum()

# Transform the table
id_columns = ['year_month']

# Reshape the data 
popular_df_time_g = pd.melt(popular_df_time_g, id_vars=id_columns, var_name='product_name', value_name='sales_volume')


IndexError: positional indexers are out-of-bounds

In [ ]:
for col in train.columns.tolist():
    print(f"Unique values in {col}: {train[col].unique()}")

In [ ]:
data_without_missing = train.dropna()

In [ ]:
data_without_missing.isnull().sum()

In [ ]:
product_list = [
    'prod_guarantees',
    'prod_current_accounts',
    'prod_derivative_account',
    'prod_payroll_account',
    'prod_junior_account',
    'prod_mas_particular_account',
    'prod_particular_account',
    'prod_particular_plus_account',
    'prod_short_term_deposits',
    'prod_medium_term_deposits',
    'prod_long_term_deposits',
    'prod_e_account',
    'prod_funds',
    'prod_mortgage',
    'prod_pensions1',
    'prod_loans',
    'prod_taxes',
    'prod_credit_card',
    'prod_securities',
    'prod_home_account',
    'prod_payroll',
    'prod_pensions2',
    'prod_direct_debit'
]

In [ ]:
len(product_list)

In [ ]:
from tqdm import tqdm
from sklearn.model_selection import train_test_split

In [ ]:
from sklearn.preprocessing import StandardScaler
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense, Concatenate, Embedding, Flatten, MultiHeadAttention, LayerNormalization
from tensorflow.keras.models import Model

# For multiple columns

In [ ]:
X = data_without_missing.loc[:, 'data_date':'segmentation']
#y = data_without_missing.loc[:, 'prod_guarantees'][:100000] for single col
y = data_without_missing.loc[:, 'prod_guarantees':'prod_direct_debit'] 
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=123)

In [ ]:
# Standardize numerical features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [ ]:
pred = {}

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense, Embedding, GlobalAveragePooling1D, Reshape
from tensorflow.keras.models import Model
from tensorflow.keras.layers import MultiHeadAttention, LayerNormalization, Dropout

# # Generate dummy numerical data
# X_train = np.random.randint(0, 100, size=(1000, 10))  # 1000 samples, 10 numerical features
# y_train = np.random.randint(0, 2, size=(1000,))  # Binary labels (0 or 1)

# Define input layer
input_layer = Input(shape=(9,), name='input_layer')

# Add embedding layer
embedding_dim = 64  # Dimension of the embedding space
vocab_size = 100  # Vocabulary size
embedding_layer = Embedding(input_dim=vocab_size, output_dim=embedding_dim)(input_layer)

# Add Reshape layer to fit Transformer's input shape
reshaped_embedding = Reshape((-1, embedding_dim))(embedding_layer)

# Add Transformer layers
num_heads = 2
dff = 32
num_layers = 2

# Multi-Head Attention
multi_head_attention = MultiHeadAttention(num_heads=num_heads, key_dim=embedding_dim)(reshaped_embedding, reshaped_embedding)
multi_head_attention = Dropout(0.1)(multi_head_attention)
multi_head_attention = LayerNormalization()(multi_head_attention + reshaped_embedding)

# Feed Forward Network
ffn = Dense(dff, activation='relu')(multi_head_attention)
ffn = Dense(embedding_dim)(ffn)
ffn = Dropout(0.1)(ffn)
transformer_output = LayerNormalization()(ffn + multi_head_attention)

# Global average pooling
pooled_output = GlobalAveragePooling1D()(transformer_output)

# Output layer
output_layer = Dense(1, activation='sigmoid')(pooled_output)

# Define model
model = Model(inputs=input_layer, outputs=output_layer)

# Compile model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])


In [ ]:
# import os

# # Define the model file name
# model_file = 'transformerm.h5'

# # Check if the model file already exists
# if os.path.exists(model_file):
#     # If the file exists, delete it
#     os.remove(model_file)

# # Save the model
# model.save(model_file)

In [ ]:
for col in product_list:
    print(col)

In [ ]:
train.info()

In [ ]:
y_test

In [ ]:
X_train.shape

In [ ]:
from tensorflow.keras.models import load_model
loaded_model = load_model("modelprod_current_accounts.h5")
print(f"Loaded model from {model_path}")

In [ ]:
product_list = [
#     'prod_guarantees',
#     'prod_current_accounts',
#     'prod_derivative_account',
#     'prod_payroll_account',
#     'prod_junior_account',
#     'prod_mas_particular_account',
#     'prod_particular_account',
#     'prod_particular_plus_account',
#     'prod_short_term_deposits',
#     'prod_medium_term_deposits',
#     'prod_long_term_deposits',
#     'prod_e_account',
#     'prod_funds',
#     'prod_mortgage',
#     'prod_pensions1',
#     'prod_loans',
#     'prod_taxes',
#     'prod_credit_card',
#     'prod_securities',
#     'prod_home_account',
#     'prod_payroll',
#     'prod_pensions2',
    'prod_direct_debit'
]

In [ ]:
import joblib

# Initialize a counter to keep track of model iterations
model_counter = 1

for col in product_list:
    print(col)
    if train[col].nunique() == 2:
        y = y_test[col]
        model.fit(X_train, y_train[col], epochs=2, batch_size=32, validation_split=0.2)
        xgb_probabilities = model.predict(X_test)
        pred[col] = xgb_probabilities
       # Save the model to the output directory
        model.save(f"/kaggle/working/model{col}.h5")
        print(f"Saved model as model{col}.h5")
        model_counter += 1
    else:
        pass


In [ ]:
from keras.models import load_model

# Replace "/kaggle/working/modelprod_payroll.h5" with the actual path to your saved model
model_path = "/kaggle/working/modelprod_payroll.h5"

loaded_model = load_model(model_path)
print(f"Loaded model from {model_path}")


In [ ]:
import numpy as np

# Example dictionary
datad = {
    'prod_guarantees': np.array([[1.4795244e-05],
                                 [1.5492973e-05],
                                 [2.2213326e-05],
                                 [2.3030307e-05],
                                 [1.4767839e-05],
                                 [1.4860431e-05]])
}

# Remove outer array brackets and flatten the arrays
for key in datad:
    datad[key] = datad[key].flatten()

# Print the modified dictionary
print(datad)


In [ ]:
for j in pred:
    print(j)

In [ ]:
for key in pred:
    pred[key] = pred[key].flatten()

In [ ]:
pred 

In [ ]:
pred_df = pd.DataFrame(pred)

data_subset = X_test

concatenated_data = pd.concat([data_subset.reset_index(drop=True), pred_df], axis=1)

In [ ]:
pred_df.head()

In [ ]:
def gbdt_product_recommender(df, row_number, top_n):
    
    prod_list = df.iloc[row_number, -len(pred_df.columns):].reset_index()
    prod_list.columns = ['product', 'pred_score']
    prod_list = prod_list.sort_values(by='pred_score', ascending=False)
    
    prod_list = prod_list[prod_list['pred_score'] >= 0.000001]
    recommend_list = prod_list.head(top_n)
    
    if recommend_list.empty:
        print("Based on the customer's info, there is no bank product recommended for now")
    
    return recommend_list

In [ ]:
gbdt_product_recommender(concatenated_data ,11,10)

In [ ]:
len(pred.values())

In [ ]:
first_values_list = []

# Iterate over the values (prediction arrays) in the pred dictionary
for predictions in pred.values():
    first_values_list.append(predictions[0])
first_values_list

In [ ]:
for i in first_values_list:
    print(len(i))

# Thompson Sampling

In [ ]:
myprob= xgb_probabilities[0]
list1=[]
for i in myprob:
    list1.append(i)
list1    

In [ ]:
import numpy as np

# Thompson Sampling function
def thompson_sampling(prod_prob, num_samples):
    rewards = np.zeros(len(prod_prob))  # Initialize rewards for each product
    selections = np.zeros(len(prod_prob))  # Initialize counts of selections for each product
    
    # Iterate for num_samples
    for _ in range(num_samples):
        # Sample from beta distribution for each product
        sampled_values = [np.random.beta(rewards[i] + 1, selections[i] - rewards[i] + 1) for i in range(len(prod_prob))]
        
        # Select the product with the highest sampled value
        selected_product = np.argmax(sampled_values)
        
        # Update rewards and selections
        reward = np.random.choice([0, 1], p=[1 - prod_prob[selected_product], prod_prob[selected_product]])
        rewards[selected_product] += reward
        selections[selected_product] += 1
    
    return rewards, selections

# Example usage
prod_prob = list1  # Example probabilities for products
num_samples = 10000  # Number of iterations for Thompson Sampling

# Run Thompson Sampling
rewards, selections = thompson_sampling(prod_prob, num_samples)

# Print results
print("Estimated rewards for each product:", rewards)
print("Number of selections for each product:", selections)

# Epsilon Greedy

In [ ]:
import numpy as np

def epsilon_greedy(prod_prob, num_samples, epsilon):
    rewards = np.zeros(len(prod_prob))  # Initialize rewards for each product
    selections = np.zeros(len(prod_prob))  # Initialize counts of selections for each product
    
    # Iterate for num_samples
    for _ in range(num_samples):
        # Epsilon-greedy action selection
        if np.random.rand() < epsilon:
            # Randomly select a product
            selected_product = np.random.choice(len(prod_prob))
        else:
            # Select the product with the highest estimated probability
            selected_product = np.argmax(rewards / (selections + 1))
        
        # Update rewards and selections
        reward = np.random.choice([0, 1], p=[1 - prod_prob[selected_product], prod_prob[selected_product]])
        rewards[selected_product] += reward
        selections[selected_product] += 1
    
    return rewards, selections

# Example usage
epsilon = 0.1  # Epsilon value for epsilon-greedy
num_samples = 10000  # Number of iterations

# Run epsilon-greedy
rewards_eg, selections_eg = epsilon_greedy(list1, num_samples, epsilon)

# Print results
print("Epsilon-Greedy: Estimated rewards for each product:", rewards_eg)
print("Epsilon-Greedy: Number of selections for each product:", selections_eg)

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# Generate x-axis values for the number of products
x_values = np.arange(len(list1))

# Plotting the oracle probabilities
plt.figure(figsize=(10, 6))
plt.bar(x_values, list1, color='blue', alpha=0.5, label='Oracle Probabilities')

plt.xlabel('Product Index')
plt.ylabel('Oracle Probability')
plt.title('Oracle Probabilities')
plt.legend()
plt.show()

In [ ]:
# Plotting the rewards obtained
plt.figure(figsize=(10, 6))
plt.bar(x_values, rewards_eg, color='red', alpha=0.5, label='Estimated Rewards')

plt.xlabel('Product Index')
plt.ylabel('Estimated Reward')
plt.title('Estimated Rewards')
plt.legend()
plt.show()

In [ ]:
# Create a list of tuples containing product name, reward, and selections
product_rewards_selections = [(product_list[i], rewards_eg[i], selections_eg[i]) for i in range(len(product_list))]

# Sort the list based on rewards in descending order
product_rewards_selections_sorted = sorted(product_rewards_selections, key=lambda x: x[1], reverse=True)

# Print the sorted list
for product, reward, selections in product_rewards_selections_sorted:
    print(f"{product}: Reward={reward}, Selections={int(selections)}")

# UCB

In [ ]:
def ucb(prod_prob, num_samples):
    rewards = np.zeros(len(prod_prob))  # Initialize rewards for each product
    selections = np.zeros(len(prod_prob))  # Initialize counts of selections for each product
    
    # Iterate for num_samples
    for t in range(1, num_samples + 1):
        # UCB action selection
        ucb_values = rewards / selections + np.sqrt(2 * np.log(t) / selections)
        selected_product = np.argmax(ucb_values)
        
        # Update rewards and selections
        reward = np.random.choice([0, 1], p=[1 - prod_prob[selected_product], prod_prob[selected_product]])
        rewards[selected_product] += reward
        selections[selected_product] += 1
    
    return rewards, selections

# Example usage
num_samples = 10000  # Number of iterations

# Run UCB
rewards_ucb, selections_ucb = ucb(list1, num_samples)

# Print results
print("UCB: Estimated rewards for each product:", rewards_ucb)
print("UCB: Number of selections for each product:", selections_ucb)

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# Generate x-axis values for the number of products
x_values = np.arange(len(list1))

# Plotting the rewards obtained by UCB
plt.figure(figsize=(10, 6))
plt.bar(x_values, rewards_ucb, color='blue', alpha=0.5, label='Estimated Rewards (UCB)')

# Adding labels and title to the plot
plt.xlabel('Product Index')
plt.ylabel('Estimated Reward')
plt.title('Estimated Rewards (UCB)')
plt.legend()
plt.show()


In [ ]:
# Create a list of tuples containing product name, reward, and selections
ucb_rewards_selections = [(product_list[i], rewards_ucb[i], selections_ucb[i]) for i in range(len(product_list))]

# Sort the list based on rewards in descending order
ucb_rewards_selections_sorted = sorted(ucb_rewards_selections, key=lambda x: x[1], reverse=True)

# Print the sorted list
for product, reward, selections in ucb_rewards_selections_sorted:
    print(f"{product}: Reward={reward}, Selections={int(selections)}")